In [1]:
## Burada hata alırsanız komut satırından "pip install findspark" komutunu çalıştırarak findspark'ı yüklemeyi unutmayın.
import findspark
findspark.init()
import pyspark # only run after findspark.init()

from pyspark import SparkConf, SparkContext
#sc = SparkContext("local","RDD-Olusturmak")

conf = SparkConf() \
.setMaster("local[4]") \
.setAppName("RDD_Olusturmak") \
.setExecutorEnv("spark.executor.memory", "4g") \
.setExecutorEnv("spark.driver.memory","2g")

sc = SparkContext(conf=conf)

## Veri Yükleme

filter(lambda x: "InvoiceNo" not in x) ile başlık satırından kurtuluyoruz

In [2]:
retailRDD = sc.textFile("C:/Users/toshiba/SkyDrive/veribilimi.co/Datasets/OnlineRetail.csv") \
.filter(lambda x: "InvoiceNo" not in x)

İlk satırı görelim başlıktan kurtulmuş muyuz?

In [3]:
retailRDD.first()

'536365;85123A;WHITE HANGING HEART T-LIGHT HOLDER;6;1.12.2010 08:26;2,55;17850;United Kingdom'

# map() dönüşümü

 Quantity ile Unit price çarparak işlem tutarını bulmak ve InvoiceNo'dan C harflerini bularak yeni bir sütunda
işlemin iptal olup olmadığını boolean olarak yazmak

map() içinde uygulayacağımız işlemleri bir fonksiyonda yazmak

In [4]:


def my_func(line):
    isCancelled = True if(line.split(";")[0].startswith("C")) else False
    total = float(line.split(";")[3]) * float(line.split(";")[5].replace(",","."))
    return (isCancelled, total)



In [5]:
retailMapPriceRDD = retailRDD.map(my_func) 
    

In [6]:
retailMapPriceRDD.take(3)

[(False, 15.299999999999999), (False, 20.34), (False, 22.0)]

İptal oanları filtreleyelim

In [7]:
retailMapPriceRDD.filter(lambda x: x[0] == True).take(10)

[(True, -27.5),
 (True, -4.65),
 (True, -19.799999999999997),
 (True, -6.959999999999999),
 (True, -6.959999999999999),
 (True, -6.959999999999999),
 (True, -41.400000000000006),
 (True, -19.799999999999997),
 (True, -39.599999999999994),
 (True, -25.5)]

İptal olanları sayalım

In [8]:
retailMapPriceRDD.filter(lambda x: x[0] == True).count()

9288

# flatMap() dönüşümü

In [9]:
retailFlatMapSplittedRDD = retailRDD.flatMap(lambda x: x.split(";"))

In [10]:
retailFlatMapSplittedRDD.count()

4335272

In [11]:
retailRDD.count()

541909

flatMap ile her kelimeyi büyük harf yapma

In [14]:
retailFlatMapUpper = retailRDD.flatMap(lambda x: x.split(";")).map(lambda x: x.upper())

In [15]:
retailFlatMapUpper.take(15)

['536365',
 '85123A',
 'WHITE HANGING HEART T-LIGHT HOLDER',
 '6',
 '1.12.2010 08:26',
 '2,55',
 '17850',
 'UNITED KINGDOM',
 '536365',
 '71053',
 'WHITE METAL LANTERN',
 '6',
 '1.12.2010 08:26',
 '3,39',
 '17850']

# İptal edilen satışların toplam tutarı

In [19]:
retailMapPriceRDD.reduceByKey(lambda x,y: x + y).take(2)

[(False, 10644560.424000263), (True, -896812.4900000116)]

In [39]:
retailMapPriceRDD.reduceByKey(lambda x,y: x + y) \
.filter(lambda x: x[0] == True) \
.map(lambda x: x[1]) \
.take(2)

[-896812.4900000116]

In [12]:
print("True") if("ile".startswith("e")) else print("False") 

False


In [17]:
"ile".replace("e","a")

'ila'

In [19]:
float("2,5".replace(",","."))

2.5